In [1]:
import numpy as np
from numba import jit, njit
import matplotlib.pyplot as plt
from random import randint
from sklearn.datasets import make_blobs
from utility import *
from tqdm import tqdm
from sklearn.cluster import k_means

In [2]:
@njit
def neighbors_clustering(sol, points, K):
    neighbors = []

    q = int(len(sol)/100)

    choices = np.arange(len(sol))
    orig_val = squared_inner_distance(sol, points, K)
    centroids = calc_centroids(sol, points, K)

    for i in range(q):
        choice = np.random.randint(len(choices))
        choices = np.delete(choices, choice)
        for k in range(K):
            new_sol = sol.copy()
            if(k != new_sol[choice]):
                new_sol[choice] = k
                new_val = orig_val - np.linalg.norm(points[choice]-centroids[sol[choice]]) + np.linalg.norm(points[choice]-centroids[k])
                neighbors.append((new_sol,new_val))


    choices1 = np.arange(len(sol))
    choices2 = np.arange(len(sol))
    for i in range(q**2):
        choice1 = np.random.randint(len(choices1))
        choices1 = np.delete(choices1, choice1)
        choice2 = np.random.randint(len(choices2))
        choices2 = np.delete(choices2, choice2)

        new_sol = sol.copy()
        new_sol[choice1], new_sol[choice2] = new_sol[choice2], new_sol[choice1]

        new_val = orig_val - np.linalg.norm(points[choice1]-centroids[sol[choice1]]) - np.linalg.norm(points[choice2]-centroids[sol[choice2]]) + np.linalg.norm(points[choice1]-centroids[sol[choice2]]) + np.linalg.norm(points[choice2]-centroids[sol[choice1]])
        neighbors.append((new_sol,new_val))
        
    return neighbors


@njit
def local_search(base_sol, points, K, verbose = True):
    old_sol = base_sol
    base_val = squared_inner_distance(old_sol, points, K)
    iter = 1
    same_sol = 0

    while True:
        neighbourhood = neighbors_clustering(old_sol, points, K)
        best_val = squared_inner_distance(old_sol, points , K)
        best_sol = old_sol

        if verbose:
            print("Iteration number:", iter, "Valore percentuale:", best_val/base_val*100, "%")

        for sol,val in neighbourhood:
            if(val < best_val):
                best_val = val
                best_sol = sol
        
        if(best_sol is old_sol):
            same_sol = same_sol + 1
            if(same_sol == 100):
                break
        else:
            same_sol = 0
            old_sol = best_sol

        iter = iter+1
    return old_sol

In [3]:
K = 10
N = 10000

points, centroids = make_blobs(n_samples=N, centers=K, n_features=2, random_state=np.random.randint(10))

sol = np.random.randint(K, size = N)
sol = local_search(sol, points, K)

print("Il valore di f.obj ottenuta è: {:.5E}".format(squared_inner_distance(sol, points, K)))

printR2sol(points, sol, K)

Iteration number: 1 Valore percentuale: 100.0 %
Iteration number: 2 Valore percentuale: 99.99753367213168 %
Iteration number: 3 Valore percentuale: 99.99364585798341 %
Iteration number: 4 Valore percentuale: 99.99144262273745 %
Iteration number: 5 Valore percentuale: 99.98943877267436 %
Iteration number: 6 Valore percentuale: 99.98684652623004 %
Iteration number: 7 Valore percentuale: 99.98375899060083 %
Iteration number: 8 Valore percentuale: 99.97979681601858 %
Iteration number: 9 Valore percentuale: 99.97619917874296 %
Iteration number: 10 Valore percentuale: 99.97231562126682 %
Iteration number: 11 Valore percentuale: 99.96768027544456 %
Iteration number: 12 Valore percentuale: 99.96513656788723 %
Iteration number: 13 Valore percentuale: 99.96375726185668 %
Iteration number: 14 Valore percentuale: 99.9608098421943 %
Iteration number: 15 Valore percentuale: 99.95852334641653 %
Iteration number: 16 Valore percentuale: 99.95443112614913 %
Iteration number: 17 Valore percentuale: 99.94

In [3]:

n_points = [500,100,200,500,1400,1000,400,700,300,800,5000,10000]
n_clusters = [7,4,3,2,3,2,4,5,4,6,4,5,10]
dim_points = [32,16,24,10,20,16,12,20,64,32,3,3]


vals = []

for test in tqdm(range(1,13)):
    points = load_points(f'C:/Users/franc/Documents/GitHub/Ricerca_Operativa_2022/Ricerca_Operativa_2022/benchmark/benchmark{test}.txt')
    N = len(points)
    K = n_clusters[test-1]
    sol = np.random.randint(K, size = N) #soluzione iniziale
    sol = local_search(sol, points, K)
    vals.append(squared_inner_distance(sol, points, K))

print(vals)
with open("risultatiLS.txt", 'w') as file:
    file.write("Local search:\n")
    file.write(str(vals))


  0%|          | 0/12 [00:00<?, ?it/s]

Iteration number: 1 Valore percentuale: 100.0 %
Iteration number: 2 Valore percentuale: 99.94143054174369 %
Iteration number: 3 Valore percentuale: 99.86104550692373 %
Iteration number: 4 Valore percentuale: 99.78860374644334 %
Iteration number: 5 Valore percentuale: 99.69983068855532 %
Iteration number: 6 Valore percentuale: 99.62535490408693 %
Iteration number: 7 Valore percentuale: 99.54637062773301 %
Iteration number: 8 Valore percentuale: 99.45333306231127 %
Iteration number: 9 Valore percentuale: 99.36372821821465 %
Iteration number: 10 Valore percentuale: 99.24262588755785 %
Iteration number: 11 Valore percentuale: 99.08357250302309 %
Iteration number: 12 Valore percentuale: 98.99720381767916 %
Iteration number: 13 Valore percentuale: 98.9178296607994 %
Iteration number: 14 Valore percentuale: 98.77293415904246 %
Iteration number: 15 Valore percentuale: 98.66408640707297 %
Iteration number: 16 Valore percentuale: 98.5729984720763 %
Iteration number: 17 Valore percentuale: 98.520

  8%|▊         | 1/12 [00:07<01:22,  7.52s/it]

 5.5437258260723645 %
Iteration number: 613 Valore percentuale: 5.5437258260723645 %
Iteration number: 614 Valore percentuale: 5.5437258260723645 %
Iteration number: 615 Valore percentuale: 5.5437258260723645 %
Iteration number: 616 Valore percentuale: 5.5437258260723645 %
Iteration number: 617 Valore percentuale: 5.5437258260723645 %
Iteration number: 618 Valore percentuale: 5.5437258260723645 %
Iteration number: 619 Valore percentuale: 5.5437258260723645 %
Iteration number: 620 Valore percentuale: 5.037774742543931 %
Iteration number: 621 Valore percentuale: 5.037774742543931 %
Iteration number: 622 Valore percentuale: 5.037774742543931 %
Iteration number: 623 Valore percentuale: 5.037774742543931 %
Iteration number: 624 Valore percentuale: 4.599354133585929 %
Iteration number: 625 Valore percentuale: 4.599354133585929 %
Iteration number: 626 Valore percentuale: 4.599354133585929 %
Iteration number: 627 Valore percentuale: 4.599354133585929 %
Iteration number: 628 Valore percentuale:

 17%|█▋        | 2/12 [00:07<00:53,  5.36s/it]

...

 3.775803016365762 %
Iteration number: 1052 Valore percentuale: 3.775803016365762 %
Iteration number: 1053 Valore percentuale: 3.775803016365762 %
Iteration number: 1054 Valore percentuale: 3.775803016365762 %
Iteration number: 1055 Valore percentuale: 3.775803016365762 %
Iteration number: 1056 Valore percentuale: 3.775803016365762 %
Iteration number: 1057 Valore percentuale: 3.775803016365762 %
Iteration number: 1058 Valore percentuale: 3.775803016365762 %
Iteration number: 1059 Valore percentuale: 3.775803016365762 %
Iteration number: 1060 Valore percentuale: 3.775803016365762 %
Iteration number: 1061 Valore percentuale: 3.775803016365762 %
Iteration number: 1062 Valore percentuale: 3.775803016365762 %
Iteration number: 1063 Valore percentuale: 3.775803016365762 %
Iteration number: 1064 Valore percentuale: 3.775803016365762 %
Iteration number: 1065 Valore percentuale: 3.775803016365762 %
Iteration number: 1066 Valore percentuale: 3.775803016365762 %
Iteration number: 1067 Valore perc

 75%|███████▌  | 9/12 [00:15<00:04,  1.42s/it]

Iteration number: 441 Valore percentuale: 5.522850389253826 %
Iteration number: 442 Valore percentuale: 5.522850389253826 %
Iteration number: 443 Valore percentuale: 5.522850389253826 %
Iteration number: 444 Valore percentuale: 5.522850389253826 %
Iteration number: 445 Valore percentuale: 5.522850389253826 %
Iteration number: 446 Valore percentuale: 5.522850389253826 %
Iteration number: 447 Valore percentuale: 5.522850389253826 %
Iteration number: 448 Valore percentuale: 5.522850389253826 %
Iteration number: 449 Valore percentuale: 4.78780162693309 %
Iteration number: 450 Valore percentuale: 4.78780162693309 %
Iteration number: 451 Valore percentuale: 4.78780162693309 %
Iteration number: 452 Valore percentuale: 4.78780162693309 %
Iteration number: 453 Valore percentuale: 4.78780162693309 %
Iteration number: 454 Valore percentuale: 4.78780162693309 %
Iteration number: 455 Valore percentuale: 4.78780162693309 %
Iteration number: 456 Valore percentuale: 4.78780162693309 %
Iteration number

 83%|████████▎ | 10/12 [00:17<00:02,  1.49s/it]

 1010 Valore percentuale: 3.4820838886047354 %
Iteration number: 1011 Valore percentuale: 3.4820838886047354 %
Iteration number: 1012 Valore percentuale: 3.4820838886047354 %
Iteration number: 1013 Valore percentuale: 3.4820838886047354 %
Iteration number: 1014 Valore percentuale: 3.4820838886047354 %
Iteration number: 1015 Valore percentuale: 3.4820838886047354 %
Iteration number: 1016 Valore percentuale: 3.4820838886047354 %
Iteration number: 1017 Valore percentuale: 3.4820838886047354 %
Iteration number: 1018 Valore percentuale: 3.4820838886047354 %
Iteration number: 1019 Valore percentuale: 3.4820838886047354 %
Iteration number: 1020 Valore percentuale: 3.4820838886047354 %
Iteration number: 1021 Valore percentuale: 3.4820838886047354 %
Iteration number: 1022 Valore percentuale: 3.4820838886047354 %
Iteration number: 1023 Valore percentuale: 3.4820838886047354 %
Iteration number: 1024 Valore percentuale: 3.4820838886047354 %
Iteration number: 1025 Valore percentuale: 3.482083888604